<a href="https://colab.research.google.com/github/GregoryG3/Thesis/blob/main/3_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [2]:
# !pip install python-docx

In [59]:
import pandas as pd
import numpy as np
import random
from datetime import datetime
from tqdm import tqdm

from google.colab import drive
from os.path import join

import re
from scipy.stats import describe

from matplotlib import pyplot as plt
import seaborn as sns

# from docx import Document
# from docx.shared import Pt
# from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
# from docx.oxml.ns import qn
# from docx.oxml import OxmlElement

# import string
# import nltk
# from nltk.util import ngrams
# from nltk.corpus import stopwords
# from collections import Counter
# from bs4 import BeautifulSoup

# from matplotlib.lines import Line2D
# from wordcloud import WordCloud
# # from PIL import Image

# Load data

In [6]:
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/Thesis/Data"
FEEDBACK_DIR = join(PROJECT_DIR, "Feedback data")
PROCESSED_DIR = join(PROJECT_DIR, "Processed")

Mounted at /content/drive


Loading the AB feedback dataset with predicted snetiments, topics and motives:

In [7]:
prediction_dataset = pd.read_csv(join(PROCESSED_DIR, "ab_predicted_2.csv"), encoding='latin1')
## finalnie trzeba pobrac plik: ab_predicted_final.csv

Loading the dataset with manually coded
 data and whole Ab_feedback for further processing:



In [13]:
manually_coded = pd.read_csv(join(PROJECT_DIR,"Annotation data/manual_coding_majority_vote_2000_texts.csv"), encoding='latin1')
ab_text_feedback = pd.read_csv(join(FEEDBACK_DIR, "ab_selected_feedback_data.csv"), encoding='latin1')

Loading the AB items

In [68]:
ab_items = pd.read_csv(join(FEEDBACK_DIR,"ab_items.csv"), encoding='latin1')

# Preprocessing

Prepare the manually coded data to be joined back in:


In [62]:
# Filter rows where global_id is in manually_coded
manually_coded_extra_info = ab_text_feedback[ab_text_feedback['global_id'].isin(manually_coded['global_id'])]

# Merge tables based on the 'global_id' column
col = ['global_id', 't_communication', 't_payment', 't_refund',
       't_price', 't_value', 't_shipping', 't_product', 't_feedback',
       't_vendor', 't_generic', 't_overall', 'pos_neg',
       'm_help_other_buyer', 'm_avoid_harm_seler', 'm_help_seller',
       'm_rew_pun_seller', 'm_reach_seller', 'm_express_emo', 'm_share_facts']

manually_coded_combi = pd.merge(manually_coded_extra_info.iloc[:, [0, 1, 3, 4, 5, 6, 7, 8]],
                                manually_coded.loc[:, col],
                                on='global_id')

# Tidy up the dates
manually_coded_combi.loc[:, 'date_left'] = pd.to_datetime(manually_coded_combi['date_left'].str.slice(stop=-6), format='%b %d, %Y', errors='coerce').dt.date


Join the prediction and manually coded subset:


In [63]:
prediction_dataset['origin'] = "prediction"
manually_coded_combi['origin'] = "manual"

# Rename the column to 'm_avoid_harm_seller' - we have a typo
manually_coded_combi = manually_coded_combi.rename(columns={'m_avoid_harm_seler': 'm_avoid_harm_seller'})

full_dataset = pd.concat([manually_coded_combi, prediction_dataset], axis=0)

Change type of columns

In [64]:
# Define the columns to convert
cols_to_convert = full_dataset.columns[8:27]

# Function to convert to Int64 if the value is finite, otherwise keep NaN
def to_int_if_finite(series):
    return series.apply(lambda x: int(x) if pd.notna(x) and np.isfinite(x) else pd.NA).astype('Int64')

# Apply the conversion to the specified columns with a progress bar
for col in tqdm(cols_to_convert, desc="Converting columns in full_dataset"):
    full_dataset[col] = to_int_if_finite(full_dataset[col])


Converting columns in full_dataset: 100%|██████████| 19/19 [00:37<00:00,  1.97s/it]


# 4 Preparing AB dataset

Merge two dataset:

In [ ]:
# filter only AB dataset
df_predict_filter = df_predict[df_predict['dataset']=='ab']
# remove duplicates from database with items
ab_items_no_duplicates = ab_items.drop_duplicates(subset=['itemid'], keep='first')

# Convert 'itemid' columns to string, handling NaN values
def is_finite(value):
    try:
        return np.isfinite(float(value))
    except (ValueError, TypeError):
        return False

mask_finite = df_predict_filter['itemid'].apply(is_finite)
df_predict_filter.loc[mask_finite, 'itemid'] = (
    df_predict_filter.loc[mask_finite, 'itemid']
    .astype(float)
    .astype(int)
    .astype(str)
)

ab_items_no_duplicates.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# Merge the entire datasets based on 'itemid'
ab_feedback = pd.merge(df_predict_filter,
                       ab_items_no_duplicates,
                       on=['itemid','seller'],
                       how='left')

ab_feedback = ab_feedback[ab_feedback['symbols_only']=='no']
ab_feedback_for_prediciton = ab_feedback.drop(columns=['symbols_only', 'dataset','buyer', 'cat2'])
ab_feedback_for_prediciton = ab_feedback_for_prediciton.dropna(subset=['category'])

<ipython-input-63-9ecb390f01f4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter.loc[mask_finite, 'itemid'] = (


In [ ]:
# # filter only AB dataset
# df_predict_filter = df_predict[df_predict['dataset']=='ab']
# # remove duplicates from database with items
# # ab_items_no_duplicates = ab_items.drop_duplicates(subset=['itemid'], keep='first')

# # Convert 'itemid' columns to string, handling NaN values
# def is_finite(value):
#     try:
#         return np.isfinite(float(value))
#     except (ValueError, TypeError):
#         return False

# mask_finite = df_predict_filter['itemid'].apply(is_finite)
# df_predict_filter.loc[mask_finite, 'itemid'] = (
#     df_predict_filter.loc[mask_finite, 'itemid']
#     .astype(float)
#     .astype(int)
#     .astype(str)
# )

# ab_items.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# df_predict_filter['price'] = pd.to_numeric(df_predict_filter['price'], errors='coerce')
# ab_items['price'] = pd.to_numeric(ab_items['price'], errors='coerce')
# # ab_items_no_duplicates.loc[:,'itemid'] = ab_items_no_duplicates['itemid'].astype(str)

# # Merge the entire datasets based on 'itemid'
# ab_feedback = pd.merge(df_predict_filter,
#                       #  ab_items_no_duplicates,
#                        ab_items,
#                        on=['itemid','seller', 'price'],
#                        how='left')

# ab_feedback = ab_feedback[ab_feedback['symbols_only']=='no']
# ab_feedback_for_prediciton = ab_feedback.drop(columns=['symbols_only', 'dataset','buyer', 'cat2'])
# ab_feedback_for_prediciton = ab_feedback_for_prediciton.dropna(subset=['category'])

<ipython-input-60-8141147f3b78>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter.loc[mask_finite, 'itemid'] = (
<ipython-input-60-8141147f3b78>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict_filter['price'] = pd.to_numeric(df_predict_filter['price'], errors='coerce')


In [ ]:
# Define a function to compare two columns, including NaNs
def compare_columns_with_nan(df, col1, col2):
    return df.apply(lambda x: x[col1] == x[col2] or (pd.isna(x[col1]) and pd.isna(x[col2])), axis=1)

# Apply the function to the DataFrame
comparison_with_nan = compare_columns_with_nan(ab_feedback_for_prediciton, 'seller_x', 'seller_y')

comparison_with_nan.value_counts()

KeyError: 'seller_x'

# Save the file

In [67]:
full_dataset.to_csv(join(PROCESSED_DIR, "full_dataset.csv"), index=False)